In [1]:
import sys
sys.path.insert(0, '../toys-data')
import conf

In [2]:
import requests

In [3]:
def slack(title, content, ip):
    response = requests.post(
        f'https://hooks.slack.com/services/{conf.SLACK_WEBHOOK_URL}',
        headers={
            'content-type': 'application/json'
        },
        json={
            'text': title,
            'blocks': [
            {
                'type': 'section',
                'text': {
                'type': 'mrkdwn',
                'text': f'{content}{ip}'
                }
            }
            ]
        }
    )
    for key, value in response.headers.items():
        print(f"{key}: {value}")

    print("\nBody:")
    print(response.text)
    print("HTTP Status Code:", response.status_code)

In [4]:
def getmyip():
    try:
        return requests.get(r'http://checkip.amazonaws.com').text.strip()
    except:
        try:
            return requests.get(r'http://api.ipify.org').text.strip()
        except:
            return ""

In [ ]:
slack("굿모닝", ":sun_with_face: :smile: :herb: ", getmyip())